In [80]:
from colorama import Fore, Style

def print_message(message_type, message):
    if message_type == "INFO":
        print(f"{Fore.YELLOW}[INFO]{Style.RESET_ALL} {message}")
    elif message_type == "ERROR":
        print(f"{Fore.RED}[ERROR]{Style.RESET_ALL} {message}")
    elif message_type == "SUCCESS":
        print(f"{Fore.GREEN}[SUCESS]{Style.RESET_ALL} {message}")
    else:
        print(f"{message}")

print_message("INFO", "This is an info")
print_message("ERROR", "This is an error")

[INFO] This is an info
[ERROR] This is an error


In [2]:
import os
import requests
from tqdm import tqdm

In [3]:
def download_file_with_progress(url, output_path):
   
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        total_size = int(response.headers.get('content-length', 0))
        with open(output_path, 'wb') as file, tqdm(
            desc=output_path,
            total=total_size,
            unit='B',
            unit_scale=True,
            unit_divisor=1024,
        ) as progress_bar:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    file.write(chunk)
                    progress_bar.update(len(chunk))
        print_message("SUCCESS", "File has been successfully downloaded.")
    else:
        print_message("ERROR", f"Something went wrong. Status code: {response.status_code}")

In [4]:
def insert_pdf_file(pdf_file_name: str) -> str:
    if pdf_file_name[-4:] != ".pdf":
        pdf_file_name += ".pdf"
    if not os.path.exists(pdf_file_name):
        print_message("INFO", "File doesn't exist, Insert Url here")
        url = input(">")

        download_file_with_progress(url, pdf_file_name)
    else:
        print_message("SUCCESS", "The file already exists")
        return pdf_file_name

In [5]:
pdf_file_name = "Hands-On Machine Learning With - Aurelien Geron"
pdf_file_name = insert_pdf_file(pdf_file_name=pdf_file_name)

[SUCESS] The file already exists


In [43]:
import fitz
from tqdm.auto import tqdm

def text_formatter(text: str) -> str:
    cleaned_text = text.replace('\n', ' ').strip()
    return cleaned_text

def open_and_read_pdf(pdf_file_name: str) -> list[dict]:
    doc = fitz.open(pdf_file_name)
    pages_and_texts = []
    for page_number, page in tqdm(enumerate(doc)):
        text = page.get_text()
        text = text_formatter(text=text)
        pages_and_texts.append({
                "page_number": page_number + 1,
                "page_char_count": len(text),
                "page_word_count": len(text.split(' ')),
                "page_sentence_count_raw": len(text.split(". ")),
                "page_token_count": len(text) / 4,
                "text": text
        })
    return pages_and_texts

In [7]:
pages_and_texts = open_and_read_pdf(pdf_file_name=pdf_file_name)

0it [00:00, ?it/s]

In [ ]:
def store_and_embed_pdf_file(pdf_file_name: str):
    pdf_file_name = insert_pdf_file(pdf_file_name=pdf_file_name)
    pages_and_texts = open_and_read_pdf(pdf_file_name_pdf_file_name)
    

In [8]:
import random 

random.sample(pages_and_texts, k=3)

[{'page_number': 689,
  'page_char_count': 1562,
  'page_word_count': 166,
  'page_sentence_count_raw': 4,
  'page_token_count': 390.5,
  'text': 'Interleaving Lines from Multiple FilesFirst, suppose you’ve loaded the California housing dataset, shuffled it(unless it was already shuffled), and split it into a training set, a validationset, and a test set. Then you split each set into many CSV files that each looklike this (each row contains eight input features plus the target median housevalue):MedInc,HouseAge,AveRooms,AveBedrms,Popul…,AveOccup,Lat…,Long…,MedianHouseValue3.5214,15.0,3.050,1.107,1447.0,1.606,37.63,-122.43,1.4425.3275,5.0,6.490,0.991,3464.0,3.443,33.69,-117.39,1.6873.1,29.0,7.542,1.592,1328.0,2.251,38.44,-122.98,1.621[...]Let’s also suppose train_filepaths contains the list of training filepaths (andyou also have valid_filepaths and test_filepaths):>>> train_filepaths[\'datasets/housing/my_train_00.csv\', \'datasets/housing/my_train_01.csv\', ...]Alternatively, you coul

## Get some more info on the data of the book

In [9]:
import pandas as pd

df = pd.DataFrame(pages_and_texts)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,1,0,1,1,0.00,
1,2,150,14,1,37.50,"Hands-On Machine Learning withScikit-Learn, Ke..."
2,3,869,93,4,217.25,"Hands-On Machine Learning with Scikit-Learn, K..."
3,4,27,4,1,6.75,October 2022: Third Edition
4,5,1150,157,5,287.50,Revision History for the Third Edition2022-10-...


In [10]:
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count
count,1351.00,1351.00,1351.00,1351.00,1351.00
mean,676.00,1284.83,192.12,6.09,321.21
std,390.14,584.26,96.68,3.94,146.06
min,1.00,0.00,1.00,1.00,0.00
25%,338.50,909.50,115.00,3.00,227.38
50%,676.00,1311.00,195.00,6.00,327.75
75%,1013.50,1749.50,267.00,8.00,437.38
max,1351.00,3066.00,506.00,33.00,766.50


## Splitting pages into sentences

In [11]:
from spacy.lang.en import English

nlp = English()

nlp.add_pipe("sentencizer")

doc = nlp("This is a sentence. This is another sentence. I like this.")
assert len(list(doc.sents)) == 3

list(doc.sents)


[This is a sentence., This is another sentence., I like this.]

In [81]:
for item in tqdm(pages_and_texts):
    item["sentences"] = list(nlp(item['text']).sents)

    item['sentences'] = [str(sentence) for sentence in item["sentences"]]
    item["page_sentence_count_spacy"] = len(item["sentences"])

  0%|          | 0/1351 [00:00<?, ?it/s]

In [13]:
random.sample(pages_and_texts, k=1)

[{'page_number': 910,
  'page_char_count': 2051,
  'page_word_count': 313,
  'page_sentence_count_raw': 11,
  'page_token_count': 512.75,
  'text': 'MaskingMaking the model ignore padding tokens is trivial using Keras: simply addmask_zero=True when creating the Embedding layer. This means thatpadding tokens (whose ID is 0) will be ignored by all downstream layers.That’s all! If you retrain the previous model for a few epochs, you will findthat the validation accuracy quickly reaches over 80%.The way this works is that the Embedding layer creates a mask tensor equalto tf.math.not_equal(inputs, 0): it is a Boolean tensor with the same shape asthe inputs, and it is equal to False anywhere the token IDs are 0, or Trueotherwise. This mask tensor is then automatically propagated by the model tothe next layer. If that layer’s call() method has a mask argument, then itautomatically receives the mask. This allows the layer to ignore theappropriate time steps. Each layer may handle the mask diff

In [14]:
df = pd.DataFrame(pages_and_texts)
df.describe().round()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_spacy
count,1351.0,1351.0,1351.0,1351.0,1351.0,1351.0
mean,676.0,1285.0,192.0,6.0,321.0,9.0
std,390.0,584.0,97.0,4.0,146.0,6.0
min,1.0,0.0,1.0,1.0,0.0,0.0
25%,338.0,910.0,115.0,3.0,227.0,4.0
50%,676.0,1311.0,195.0,6.0,328.0,8.0
75%,1014.0,1750.0,267.0,8.0,437.0,12.0
max,1351.0,3066.0,506.0,33.0,766.0,52.0


### Making chunks from the text 

In [15]:
num_sentence_chunk_size = 10 

def split_list(input_list: list, slice_size: int =num_sentence_chunk_size) -> list[str]:
    return [input_list[i:i+slice_size + 1] for i in range(0, len(input_list), slice_size)]

In [16]:
tl = list(range(25))
split_list(tl)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
 [20, 21, 22, 23, 24]]

In [17]:
for item in tqdm(pages_and_texts):
    item["sentence_chunks"] = split_list(input_list=item["sentences"],
                                        slice_size=num_sentence_chunk_size)
    item["num_chunks"] = len(item["sentence_chunks"])

  0%|          | 0/1351 [00:00<?, ?it/s]

In [18]:
random.sample(pages_and_texts, k=2)

[{'page_number': 1007,
  'page_char_count': 1828,
  'page_word_count': 250,
  'page_sentence_count_raw': 7,
  'page_token_count': 457.0,
  'text': 'Let’s start building a variational autoencoder for Fashion MNIST (as shownin Figure 17-11, but using the γ tweak). First, we will need a custom layer tosample the codings, given μ and γ:class Sampling(tf.keras.layers.Layer):    def call(self, inputs):        mean, log_var = inputs        return tf.random.normal(tf.shape(log_var)) * tf.exp(log_var / 2) + meanThis Sampling layer takes two inputs: mean (μ) and log_var (γ). It uses thefunction tf.random.normal() to sample a random vector (of the same shape asγ) from the Gaussian distribution, with mean 0 and standard deviation 1.Then it multiplies it by exp(γ / 2) (which is equal to σ, as you can verifymathematically), and finally it adds μ and returns the result. This samples acodings vector from the Gaussian distribution with mean μ and standarddeviation σ.Next, we can create the encoder, usi

In [19]:
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_spacy,num_chunks
count,1351.00,1351.00,1351.00,1351.00,1351.00,1351.00,1351.00
mean,676.00,1284.83,192.12,6.09,321.21,8.68,1.38
std,390.14,584.26,96.68,3.94,146.06,5.71,0.58
min,1.00,0.00,1.00,1.00,0.00,0.00,0.00
25%,338.50,909.50,115.00,3.00,227.38,4.00,1.00
50%,676.00,1311.00,195.00,6.00,327.75,8.00,1.00
75%,1013.50,1749.50,267.00,8.00,437.38,12.00,2.00
max,1351.00,3066.00,506.00,33.00,766.50,52.00,6.00


In [20]:
import re

pages_and_chunks = []
for i in tqdm(pages_and_texts):
    for sentence_chunk in i['sentence_chunks']:
        chunk_dict = {}
        chunk_dict["page_number"] = i["page_number"]
        joined_sentence_chunk = "".join(sentence_chunk).replace("  ", " ").strip()
        joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk)
        chunk_dict["sentence_chunk"] = joined_sentence_chunk
        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
        chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4
        pages_and_chunks.append(chunk_dict)
    
len(pages_and_chunks)

  0%|          | 0/1351 [00:00<?, ?it/s]

1860

In [21]:
random.sample(pages_and_chunks, k=2)

[{'page_number': 267,
  'sentence_chunk': "With stochastic and mini-batch gradient descent, the curves are not so smooth, and it maybe hard to know whether you have reached the minimum or not. One solution is to stoponly after the validation error has been above the minimum for some time (when you areconfident that the model will not do any better), then roll back the model parameters to thepoint where the validation error was at a minimum. Here is a basic implementation of early stopping:from copy import deepcopyfrom sklearn.metrics import mean_squared_errorfrom sklearn.preprocessing import StandardScalerX_train, y_train, X_valid, y_valid = [...] # split the quadratic datasetpreprocessing = make_pipeline(PolynomialFeatures(degree=90, include_bias=False),               StandardScaler())X_train_prep = preprocessing.fit_transform(X_train)X_valid_prep = preprocessing.transform(X_valid)sgd_reg = SGDRegressor(penalty=None, eta0=0.002, random_state=42)n_epochs = 500best_valid_rmse = float('i

In [22]:
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,1860.00,1860.00,1860.00,1860.00
mean,676.23,962.78,141.11,240.70
std,376.24,562.09,85.76,140.52
min,2.00,3.00,1.00,0.75
25%,360.75,492.75,73.00,123.19
50%,680.00,995.50,131.00,248.88
75%,996.00,1411.00,210.00,352.75
max,1351.00,3052.00,492.00,763.00


## Filter out chunks with less than 20 tokens

In [23]:
min_token_lenght = 20
for row in df[df["chunk_token_count"] <= min_token_lenght].sample(5).iterrows():
    print(f"Chunk token count: {row[1]['chunk_token_count']} | Text: {row[1]['sentence_chunk']}")

Chunk token count: 19.25 | Text: Figure 19-6. Executing a TensorFlow graph across multiple devices in parallel
Chunk token count: 11.0 | Text: the data. For now, let’s go with this model.
Chunk token count: 6.5 | Text: How muchdoes PCA help now?
Chunk token count: 19.25 | Text: Now let’s look at the second common building block of CNNs: the poolinglayer.
Chunk token count: 14.75 | Text: Tensor: shape=(2, 3), dtype=int64, numpy= array([[0, 1, 2],


In [24]:
pages_and_chunks_over_min_token_len = df[df["chunk_token_count"] > min_token_lenght].to_dict(orient="records")

In [25]:
random.sample(pages_and_chunks_over_min_token_len, k=1)

[{'page_number': 937,
  'sentence_chunk': 'Figure 16-8. The original 2017 transformer architecture\u2060If you use the transformer for NMT, then during training you must feed the22',
  'chunk_char_count': 132,
  'chunk_word_count': 20,
  'chunk_token_count': 33.0}]

## Embedding chunks

In [26]:
test_sentences = ["Hello This is a test for the embedding model",
                 "this is a second sentence for the model",
                 "The sky is blue"]

In [27]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2",
                                     device="cuda")


embeddings = embedding_model.encode(test_sentences,
                                    batch_size=32,
                                    convert_to_tensor=True)


In [28]:
sentences_and_embeddings = dict(zip(test_sentences, embeddings))

In [29]:
sentences_and_embeddings

{'Hello This is a test for the embedding model': tensor([-5.1741e-04, -8.9576e-02, -4.3483e-02,  6.3491e-02, -1.5166e-02,
          3.6753e-02,  5.0416e-02,  2.7074e-02,  4.1897e-02,  1.8115e-03,
          6.2513e-02, -9.2316e-03,  2.6844e-03,  5.2567e-02,  3.8520e-02,
         -4.6451e-02,  5.5088e-02,  2.7253e-02, -9.0316e-03, -1.0031e-02,
         -1.9533e-02, -5.2470e-02,  4.5585e-03,  1.2949e-02, -1.9578e-02,
          2.3228e-02, -2.3299e-02,  5.9363e-03, -7.0727e-03, -5.1463e-02,
         -1.7793e-02,  2.3849e-02,  5.1635e-02, -3.2830e-02,  1.4976e-06,
         -5.8171e-02, -1.4653e-02,  6.8788e-03, -8.3841e-03,  1.3590e-02,
          1.0240e-02,  6.2663e-02, -7.4270e-03,  2.7005e-02, -2.4735e-02,
         -1.4349e-02, -1.6881e-02,  4.8620e-02, -1.3179e-02,  5.7703e-02,
          6.5349e-03,  2.2798e-03, -2.4195e-02, -3.1098e-02,  5.1947e-02,
          2.4326e-02,  3.8770e-02,  3.3131e-02, -6.9657e-03,  3.5400e-02,
         -3.7303e-02,  1.6646e-02,  2.4014e-02,  3.6676e-02,  4.

In [31]:
%%time
for item in tqdm(pages_and_chunks_over_min_token_len):
    item["embedding"] = embedding_model.encode(item["sentence_chunk"])

  0%|          | 0/1765 [00:00<?, ?it/s]

CPU times: user 4min 20s, sys: 4.67 s, total: 4min 25s
Wall time: 53.6 s


# TODO
## Creating a chromadb client for storing embeddings

> **NOTE** maybe not
>
> [INFO] Time taken to get scores on 1765000 embeddings: 0.00286 seconds.

In [24]:
import chromadb
from chromadb.utils import embedding_functions
chroma_client = chromadb.Client()

In [33]:
collection = chroma_client.create_collection(name="embeddings")

In [41]:
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-mpnet-base-v2")

In [ ]:
# results = collection.query(
#     query_texts=["This is a query document about hawaii"], # Chroma will embed this for you
#     n_results=2 # how many results to return
# )
# print(results)

## adding the embeddings to the chunks

In [54]:
%%time

text_chunks = [item["sentence_chunk"] for item in pages_and_chunks_over_min_token_len]
random.sample(text_chunks, k=3)

CPU times: user 661 μs, sys: 19 μs, total: 680 μs
Wall time: 690 μs


['The CNN applies an overall stride of 32 to theinput image (i.e., if you add up all the strides greater than 1), meaning the lastlayer outputs feature maps that are 32 times smaller than the input image. This is clearly too coarse, so they added a single upsampling layer thatmultiplies the resolution by 32. There are several solutions available for upsampling (increasing the size of an',
 'tf.keras.layers. TextVectorization, Text Preprocessing-TextPreprocessing, Creating the Training Dataset, Building and Training theChar-RNN Model, Sentiment Analysis, Sentiment Analysis-AnEncoder–Decoder Network for Neural Machine Translationtf.keras.layers. TimeDistributed, Forecasting Using a Sequence-to-Sequence Modeltf.keras.losses. Huber, Custom Loss Functionstf.keras.losses.kullback_leibler_divergence(), Sparse Autoencoderstf.keras.losses. Loss, Saving and Loading Models That Contain CustomComponentstf.keras.losses.sparse_categorical_crossentropy(), Compiling the model,CNN Architectures, Buildi

In [23]:
# Embed all the text in batches

text_chunks_embeddings = embedding_model.encode(text_chunks,
                                               batch_size=32,
                                               convert_to_tensor=True)
text_chunks_embeddings

NameError: name 'text_chunks' is not defined

### Saving embeddings into a file (Temp)

In [22]:
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_over_min_token_len)
embeddings_df_save_path = f"embedding/{pdf_file_name}.csv"
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index=False)

NameError: name 'pages_and_chunks_over_min_token_len' is not defined

In [87]:
# reading the csv 

text_chunks_and_embedding_df_load = pd.read_csv(embeddings_df_save_path)
text_chunks_and_embedding_df_load.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,2,"Hands-On Machine Learning withScikit-Learn, Ke...",150,14,37.50,[-2.78972145e-02 -1.74421235e-03 -4.81583700e-...
1,3,"Hands-On Machine Learning with Scikit-Learn, K...",873,97,218.25,[ 2.48818621e-02 6.14474006e-02 -5.22879586e-...
2,5,Revision History for the Third Edition2022-10-...,1152,159,288.00,[ 9.69696417e-03 -2.23619733e-02 -4.10474986e-...
3,7,"The Machine Learning TsunamiIn 2006, Geoffrey ...",1173,171,293.25,[-3.19831981e-03 1.02457978e-01 -5.40223382e-...
4,8,"Machine Learning in Your ProjectsSo, naturally...",867,131,216.75,[ 4.60035866e-03 9.44859162e-02 -7.24032298e-...


In [91]:
text_chunks_and_embedding_df_load["sentence_chunk"].iloc[357]

'Similarity FeaturesAnother technique to tackle nonlinear problems is to add features computedusing a similarity function, which measures how much each instanceresembles a particular landmark, as we did in Chapter 2 when we added thegeographic similarity features. For example, let’s take the 1D dataset fromearlier and add two landmarks to it at x = –2 and x = 1 (see the left plot inFigure 5-8). Next, we’ll define the similarity function to be the Gaussian RBFwith γ = 0.3. This is a bell-shaped function varying from 0 (very far awayfrom the landmark) to 1 (at the landmark). Now we are ready to compute the new features. For example, let’s look at theinstance x = –1: it is located at a distance of 1 from the first landmark and 2from the second landmark. Therefore, its new features are x = exp(–0.3 × 1 )≈ 0.74 and x = exp(–0.3 × 2 ) ≈ 0.30. The plot on the right in Figure 5-8shows the transformed dataset (dropping the original features). As you cansee, it is now linearly separable. Figure 

# Rag - Search and Answer

### Similarity search

In [1]:
from colorama import Fore, Style

def print_message(message_type, message):
    if message_type == "INFO":
        print(f"{Fore.YELLOW}[INFO]{Style.RESET_ALL} {message}")
    elif message_type == "ERROR":
        print(f"{Fore.RED}[ERROR]{Style.RESET_ALL} {message}")
    elif message_type == "SUCCESS":
        print(f"{Fore.GREEN}[SUCESS]{Style.RESET_ALL} {message}")
    else:
        print(f"{message}")

print_message("INFO", "This is an info")
print_message("ERROR", "This is an error")

[INFO] This is an info
[ERROR] This is an error


In [2]:
import random

import torch
import numpy as np
import pandas as pd
from tqdm import tqdm

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Import texts and embedding df
text_chunks_and_embedding_df = pd.read_csv("Hands-On Machine Learning With - Aurelien Geron.pdf.csv")

# Convert embedding column back to np.array (it got converted to string when it got saved to CSV)
text_chunks_and_embedding_df["embedding"] = text_chunks_and_embedding_df["embedding"].apply(lambda x: np.fromstring(x.strip("[]"), sep=" "))

# Convert texts and embedding df to list of dicts
pages_and_chunks = text_chunks_and_embedding_df.to_dict(orient="records")

# Convert embeddings to torch tensor and send to device (note: NumPy arrays are float64, torch tensors are float32 by default)
embeddings = torch.tensor(np.array(text_chunks_and_embedding_df["embedding"].tolist()), dtype=torch.float32).to(device)
embeddings.shape

torch.Size([1765, 768])

In [4]:
embeddings

tensor([[-0.0279, -0.0017, -0.0482,  ...,  0.0007, -0.0182, -0.0057],
        [ 0.0249,  0.0614, -0.0523,  ...,  0.0169, -0.0166, -0.0051],
        [ 0.0097, -0.0224, -0.0410,  ..., -0.0104, -0.0108, -0.0204],
        ...,
        [ 0.0017,  0.0756, -0.0428,  ...,  0.0299,  0.0373, -0.0241],
        [-0.0071,  0.0351, -0.0043,  ...,  0.0399,  0.0262, -0.0321],
        [ 0.0198,  0.0750, -0.0211,  ...,  0.0131, -0.0042, -0.0147]],
       device='cuda:0')

In [5]:
# creeating the model, this is just used if you havent already run the model above

from sentence_transformers import util, SentenceTransformer


embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2", 
                                      device=device)

/home/smitsis/rag_from_scratch/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


#### TODO: semantic search pipeline


1. Define a query string.
2. Turn the query string into an embedding
3. Perform a dot product or cosine similarity function between the text embedding and the query embedding
4. Sort the results from k in descending order

In [6]:
query = "Ridge regression"
print(f"Query: {query}")

# embed query
query_embedding = embedding_model.encode(query, convert_to_tensor=True)


# Get similarity scores with dot product (use cosine similarity if outputs are not normalized)

from time import perf_counter as timer

start_time = timer()
dot_scores = util.dot_score(a=query_embedding, b=embeddings)[0]
end_time = timer()

print_message("INFO", f"Time taken to get scores on {len(embeddings)} embeddings: {end_time - start_time:.5f} seconds.")

# 4 get top-k results

top_results_dot_product = torch.topk(dot_scores, k=100)
top_results_dot_product

Query: Ridge regression
[INFO] Time taken to get scores on 1765 embeddings: 0.00078 seconds.


torch.return_types.topk(
values=tensor([0.6800, 0.6272, 0.6144, 0.6123, 0.6080, 0.5475, 0.5370, 0.5321, 0.5214,
        0.5082, 0.5047, 0.4858, 0.4855, 0.4808, 0.4768, 0.4709, 0.4702, 0.4683,
        0.4650, 0.4572, 0.4565, 0.4548, 0.4508, 0.4493, 0.4492, 0.4465, 0.4458,
        0.4445, 0.4379, 0.4372, 0.4280, 0.4275, 0.4275, 0.4272, 0.4259, 0.4257,
        0.4249, 0.4243, 0.4229, 0.4199, 0.4197, 0.4187, 0.4175, 0.4170, 0.4167,
        0.4134, 0.4134, 0.4126, 0.4111, 0.4110, 0.4093, 0.4087, 0.4055, 0.4054,
        0.4037, 0.4026, 0.4021, 0.4020, 0.4014, 0.4010, 0.3998, 0.3974, 0.3938,
        0.3927, 0.3901, 0.3892, 0.3884, 0.3853, 0.3839, 0.3807, 0.3799, 0.3799,
        0.3795, 0.3793, 0.3774, 0.3771, 0.3764, 0.3719, 0.3700, 0.3679, 0.3677,
        0.3655, 0.3650, 0.3641, 0.3638, 0.3618, 0.3612, 0.3611, 0.3610, 0.3605,
        0.3603, 0.3596, 0.3592, 0.3590, 0.3576, 0.3576, 0.3576, 0.3573, 0.3569,
        0.3562], device='cuda:0'),
indices=tensor([ 309,  312,  341,  311,  310,  315, 1

In [7]:
text_chunks_and_embedding_df["sentence_chunk"].iloc[309]

'Ridge RegressionRidge regression (also called Tikhonov regularization) is a regularizedversion of linear regression: a regularization term equal to αm∑i=1nθi2 isadded to the MSE. This forces the learning algorithm to not only fit the databut also keep the model weights as small as possible. Note that theregularization term should only be added to the cost function during training. Once the model is trained, you want to use the unregularized MSE (or theRMSE) to evaluate the model’s performance. The hyperparameter α controls how much you want to regularize the model. If α = 0, then ridge regression is just linear regression. If α is very large, thenall weights end up very close to zero and the result is a flat line going throughthe data’s mean. Equation 4-8 presents the ridge regression cost function.\u2060Equation 4-8. Ridge regression cost functionJ(θ)=MSE(θ)+αm∑i=1nθi2Note that the bias term θ is not regularized (the sum starts at i = 1, not 0). Ifwe define w as the vector of feature

In [8]:
dot = torch.dot(embeddings[309], query_embedding)
print(f"just dot prod {dot:.4f}")
dot = dot / (torch.sqrt(torch.sum(embeddings[309] ** 2)) *  torch.sqrt(torch.sum(query_embedding ** 2))) 
print(f"cosine similarity {dot:.4f}")


just dot prod 0.6800
cosine similarity 0.6800


In [9]:
(torch.sqrt(torch.sum(embeddings[309] ** 2)) *  torch.sqrt(torch.sum(query_embedding ** 2))) 

tensor(1., device='cuda:0')

In [10]:
larger_embeddings = torch.rand(1000*embeddings.shape[0], 768).to(device)
print(f'Embeddings shape {larger_embeddings.shape}')

Embeddings shape torch.Size([1765000, 768])


In [13]:
start_time = timer()
dot_scores = util.dot_score(a=query_embedding, b=larger_embeddings)[0]
end_time = timer()
# print(dot_scores.shape)

print_message("INFO", f"Time taken to get scores on {len(larger_embeddings)} embeddings: {end_time - start_time:.5f} seconds.")


top_results_dot_product = torch.topk(dot_scores, k=100)
top_results_dot_product

[INFO] Time taken to get scores on 1765000 embeddings: 0.00286 seconds.


torch.return_types.topk(
values=tensor([1.5886, 1.5866, 1.5200, 1.5178, 1.5173, 1.5087, 1.4967, 1.4873, 1.4778,
        1.4660, 1.4611, 1.4571, 1.4494, 1.4467, 1.4458, 1.4402, 1.4304, 1.4287,
        1.4246, 1.4229, 1.4207, 1.4195, 1.4176, 1.4146, 1.4123, 1.4107, 1.4094,
        1.4039, 1.4033, 1.4002, 1.3973, 1.3921, 1.3899, 1.3880, 1.3832, 1.3825,
        1.3814, 1.3801, 1.3792, 1.3791, 1.3785, 1.3779, 1.3764, 1.3748, 1.3731,
        1.3728, 1.3727, 1.3723, 1.3668, 1.3668, 1.3634, 1.3628, 1.3623, 1.3604,
        1.3603, 1.3602, 1.3599, 1.3587, 1.3586, 1.3580, 1.3570, 1.3570, 1.3568,
        1.3568, 1.3565, 1.3562, 1.3554, 1.3546, 1.3544, 1.3494, 1.3489, 1.3487,
        1.3463, 1.3428, 1.3414, 1.3387, 1.3382, 1.3376, 1.3355, 1.3342, 1.3330,
        1.3330, 1.3329, 1.3314, 1.3294, 1.3279, 1.3264, 1.3262, 1.3236, 1.3231,
        1.3214, 1.3192, 1.3174, 1.3167, 1.3154, 1.3141, 1.3140, 1.3137, 1.3130,
        1.3127], device='cuda:0'),
indices=tensor([ 721033,  317068,  700443, 1130865, 1

### Implementing a Re-Ranker

- Re-rank the top k=100 results
- Select the top=5 results